<a href="https://colab.research.google.com/github/kobeisfree94/Projects/blob/master/Deep_Learning/Natural_Language_Processing/Machine_Translation/Machine_Translation_Project_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import string
import re
import numpy as np
from numpy import array, argmax, random, take
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Masking
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import drive

drive.mount('/content/drive')

eng_kor = '/content/drive/My Drive/Data AI_BootCamp/DataSets/kor.txt'


data = pd.read_csv(eng_kor, sep='\t', header=None)
data.columns=['eng', 'kor', 'delete']
data.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,eng,kor,delete
0,Go.,가.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,안녕.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run!,뛰어!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run.,뛰어.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Who?,누구?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


Preprocessing

In [3]:
#Check for Missing Values
data.isna().sum()

eng       0
kor       0
delete    0
dtype: int64

In [4]:
#Lowercase
data['eng']= data['eng'].apply(lambda x: x.lower())
data['kor']= data['kor'].apply(lambda x: x.lower())

In [5]:
data= data[['eng', 'kor']]
data.shape

(3798, 2)

In [6]:
num_samples = 3798

In [7]:
#Regex
regex = r"[^a-zA-Z0-9가-힣 ]"

#Substitute
subst = ""


In [8]:
def preprocess(text):

    tokens = re.sub(regex, subst, text)
    
    return tokens

In [9]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open(eng_kor, 'r') as lines:
    for i, line in enumerate(lines):
      src_line, tar_line, _ = line.strip().split('\t')

      src_line = [w for w in preprocess(src_line).split()]

      tar_line = preprocess(tar_line)
      tar_line_in = [w for w in ('<sos> ' + tar_line).split()]
      tar_line_out = [w for w in (tar_line + ' <eos>').split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break
  return encoder_input, decoder_input, decoder_target

In [10]:
sents_en_in, sents_kor_in, sents_kor_out = load_preprocessed_data()
print('Encoder Input :',sents_en_in[:5])
print('Decoder Input :',sents_kor_in[:5])
print('Decoder Label/Target :',sents_kor_out[:5])

Encoder Input : [['Go'], ['Hi'], ['Run'], ['Run'], ['Who']]
Decoder Input : [['<sos>', '가'], ['<sos>', '안녕'], ['<sos>', '뛰어'], ['<sos>', '뛰어'], ['<sos>', '누구']]
Decoder Label/Target : [['가', '<eos>'], ['안녕', '<eos>'], ['뛰어', '<eos>'], ['뛰어', '<eos>'], ['누구', '<eos>']]


Tokenizer/Embedding/Padding

In [11]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_kor = Tokenizer(filters="", lower=False)
tokenizer_kor.fit_on_texts(sents_kor_in)
tokenizer_kor.fit_on_texts(sents_kor_out)

decoder_input = tokenizer_kor.texts_to_sequences(sents_kor_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_kor.texts_to_sequences(sents_kor_out)
decoder_target = pad_sequences(decoder_target, padding="post")


print('Encoder Input Shape :', encoder_input.shape)
print('Decoder Input Shape :', decoder_input.shape)
print('Decoder Label/Target Shape :', decoder_target.shape)

Encoder Input Shape : (3798, 101)
Decoder Input Shape : (3798, 90)
Decoder Label/Target Shape : (3798, 90)


In [12]:
#Find Vocab Size
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_kor.word_index) + 1
print("English Vocab Size : {:d}, Korean Vocab size : {:d}".format(src_vocab_size, tar_vocab_size))

English Vocab Size : 2878, Korean Vocab size : 5769


In [13]:
#Index src/tar in dictionary form--- for evaluation
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_kor.word_index
index_to_tar = tokenizer_kor.index_word

In [14]:
#Shuffle Data
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('Random Sequence :',indices)

Random Sequence : [3343  734 3509 ...  898  980 2952]


In [15]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [16]:
val = int(num_samples*0.1)
print('Number of Validation Set: ', val)

Number of Validation Set:  379


In [17]:
encoder_input_train = encoder_input[:-val]
decoder_input_train = decoder_input[:-val]
decoder_target_train = decoder_target[:-val]

encoder_input_test = encoder_input[-val:]
decoder_input_test = decoder_input[-val:]
decoder_target_test = decoder_target[-val:]

In [18]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(3419, 101)
(3419, 90)
(3419, 90)
(379, 101)
(379, 90)
(379, 90)


Model

In [19]:
embedding_dim = 64
hidden_units = 64

In [20]:
#Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [21]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [22]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train,
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
27/27 [==============================] - 14s 154ms/step - loss: 7.9112 - acc: 0.8709 - val_loss: 6.0955 - val_acc: 0.9420
Epoch 2/50
27/27 [==============================] - 2s 60ms/step - loss: 3.7793 - acc: 0.9412 - val_loss: 1.5201 - val_acc: 0.9420
Epoch 3/50
27/27 [==============================] - 2s 60ms/step - loss: 0.8672 - acc: 0.9412 - val_loss: 0.5749 - val_acc: 0.9420
Epoch 4/50
27/27 [==============================] - 2s 60ms/step - loss: 0.5346 - acc: 0.9412 - val_loss: 0.5007 - val_acc: 0.9420
Epoch 5/50
27/27 [==============================] - 2s 60ms/step - loss: 0.4877 - acc: 0.9412 - val_loss: 0.4729 - val_acc: 0.9420
Epoch 6/50
27/27 [==============================] - 2s 60ms/step - loss: 0.4642 - acc: 0.9412 - val_loss: 0.4563 - val_acc: 0.9420
Epoch 7/50
27/27 [==============================] - 2s 60ms/step - loss: 0.4482 - acc: 0.9412 - val_loss: 0.4431 - val_acc: 0.9420
Epoch 8/50
27/27 [==============================] - 2s 61ms/step - loss: 0.4350 -

In [23]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [24]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [25]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [30]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : Tom has a learning disorder 
정답문장 : 톰은 학습 장애가 있어 
번역문장 : 톰은 톰이 좀 있어 
--------------------------------------------------
입력문장 : Im taking a shower right now 
정답문장 : 난 지금 샤워하고 있어 
번역문장 : 톰은 톰이 좀 있어 
--------------------------------------------------
입력문장 : Tom shrugged 
정답문장 : 톰이 어깨를 으쓱했어 
번역문장 : 톰은 톰이 좀 있어 
--------------------------------------------------
입력문장 : Tom is taking baby steps 
정답문장 : 톰은 걸음마를 하고 있어 
번역문장 : 톰은 톰이 좀 있어 
--------------------------------------------------
입력문장 : Tom has no social skills 
정답문장 : 톰은 눈치가 없어 
번역문장 : 톰은 톰이 좀 있어 
--------------------------------------------------
